In [8]:
! pip install -U "transformers[torch]" pypdf python-docx langdetect sentencepiece sacremoses

In [13]:
import gradio as gr
from pypdf import PdfReader
import docx
import os
from langdetect import detect
from huggingface_hub import HfApi
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline



In [14]:
summarizer = pipeline(task="summarization", model="facebook/bart-large-cnn")

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

translator_to_french = pipeline(
    task="translation_en_to_fr", model="Helsinki-NLP/opus-mt-en-fr"
)
translator_to_english = pipeline(
    task="translation_fr_to_en", model="Helsinki-NLP/opus-mt-fr-en"
)

MAX_FILE_SIZE = 10000000

In [11]:
class TextExtractor:
    def __init__(self, doc_location: str):
        if doc_location is None:
            raise Exception(f"Please select a PDF to summarize")
        self.doc_location = doc_location

    def extract_text_from_pdf(self):
        reader = PdfReader(self.doc_location)
        text = ""

        for page in reader.pages:
            text += page.extract_text()

        return text

    def extract_text_from_doc(self):
        doc = docx.Document(self.doc_location)
        text = ""

        for paragraph in doc.paragraphs:
            text += paragraph.text + "\n"
        return text

    def extract_text_from_txt(self):
        with open(self.doc_location, "r", encoding="utf-8") as file:
            text = file.read()
        return text

    def extract_text_from_txt(self):
        with open(self.doc_location, "r", encoding="utf-8") as file:
            text = file.read()
        return text
    
    def text_length(self):
        words = self.text.split()
        num_words = len(words)
        return num_words

    def get_text(self) -> str:
        file_extension = os.path.splitext(self.doc_location)[1]
        if file_extension == ".pdf":
            self.text = self.extract_text_from_pdf()
        elif file_extension == ".txt":
            self.text = self.extract_text_from_txt()
        elif file_extension == ".docx" or file_extension == ".doc":
            self.text = self.extract_text_from_doc()
        else:
            raise gr.Error(f"We only support .pdf, .txt, .doc and .docx files")

        if len(self.text) > MAX_FILE_SIZE:
            raise gr.Error(
                f"Document exceeds the maximum supported size of {MAX_FILE_SIZE} characters."
            )

        return self.text

In [22]:
text_extractor = TextExtractor("data/25-PAGE-FRENCH-TEXT.pdf")
text = text_extractor.get_text()

text_length = text_extractor.text_length()
summary_length = int(text_length / 2)

try:
  summary = summarizer(text, max_length=summary_length, do_sample=False)[0]["summary_text"]
except Exception as ex:
  max_length = tokenizer.model_max_length
  inputs = tokenizer(text, truncation=True, max_length=max_length, return_tensors="pt")

  summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=summary_length, early_stopping=True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
%%time
text_extractor = TextExtractor("data/25-PAGE-FRENCH-TEXT.pdf")
text_extractor = TextExtractor("data/pd-file-example.pdf")
text = text_extractor.get_text()

text_length = text_extractor.text_length()
summary_length = int(text_length / 2)


In [23]:
detected_lang = detect(summary)
detected_lang

'fr'

## Update space

In [28]:
token = os.getenv("HUGGINFACE_TAKEN")
api = HfApi(token=token)
api.upload_file(
    path_or_fileobj="requirements.txt",
    path_in_repo="requirements.txt",
    repo_id="noahnsimbe/text-summarizer",
    repo_type="space",
)
api.upload_file(
    path_or_fileobj="app.py",
    path_in_repo="app.py",
    repo_id="noahnsimbe/text-summarizer",
    repo_type="space",
)

CommitInfo(commit_url='https://huggingface.co/spaces/noahnsimbe/text-summarizer/commit/81226c4cce482f0b896b563fbd8ff779f5f576d2', commit_message='Upload app.py with huggingface_hub', commit_description='', oid='81226c4cce482f0b896b563fbd8ff779f5f576d2', pr_url=None, pr_revision=None, pr_num=None)